In [29]:
import os 
import pandas as pd
import numpy as np
import datetime as dt
# import ee
# import geemap as geemap
from IPython.display import HTML

import panel as pn
import param
import plotly.express as px
import plotly.graph_objects as go

pn.extension('plotly')

In [30]:
%load_ext google.cloud.bigquery

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'gbif-challenge-deed5b20a659.json' ##change this

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [31]:
from google.cloud import bigquery

client = bigquery.Client()

In [ ]:
# %%bigquery df_bq
# SELECT
#     *
# FROM `gbif-challenge.airflow_uploads.gbif_combined`
# WHERE DATE(eventDate) = "2022-04-04"
# ee.Initialize()

In [32]:
df = pd.read_csv('human_interference_sample.csv')
df.head()

,key,publishingCountry,basisOfRecord,scientificName,species,genericName,iucnRedListCategory,decimalLongitude,decimalLatitude,coordinateUncertaintyInMeters,eventDate,recordedBy,country,countryCode,date,is_invasive,avg_radiance,avg_deg_urban
0,3031987049,US,HUMAN_OBSERVATION,"Chrysocyon brachyurus (Illiger, 1815)",Chrysocyon brachyurus,Chrysocyon,NT,-47.491164,-22.258903,30298.0,2021-01-13T05:07:00,registroscomunitariosderioclarosp,Brazil,BR,2021-01-13,False,1.463336,0.389394
1,3307456876,US,HUMAN_OBSERVATION,"Chrysocyon brachyurus (Illiger, 1815)",Chrysocyon brachyurus,Chrysocyon,NT,-43.628159,-17.983999,587863.0,2021-06-17T13:21:01,Enrico A. R. Tosto,Brazil,BR,2021-06-17,False,0.234009,0.352468
2,3328022769,US,HUMAN_OBSERVATION,"Chrysocyon brachyurus (Illiger, 1815)",Chrysocyon brachyurus,Chrysocyon,NT,-47.509019,-22.525994,30278.0,2021-07-09T17:33:00,mariosoares,Brazil,BR,2021-07-09,False,5.220234,0.820176
3,3335201455,US,HUMAN_OBSERVATION,"Chrysocyon brachyurus (Illiger, 1815)",Chrysocyon brachyurus,Chrysocyon,NT,-46.934391,-21.306569,30396.0,2021-07-18T06:34:00,CRISTINA RAPPA,Brazil,BR,2021-07-18,False,1.010773,0.391316
4,3773393229,US,HUMAN_OBSERVATION,"Chrysocyon brachyurus (Illiger, 1815)",Chrysocyon brachyurus,Chrysocyon,NT,-46.874809,-21.219875,30396.0,2021-07-18T19:34:00,ademircarosia,Brazil,BR,2021-07-18,False,0.943412,0.683220


In [33]:
#### all the filter widgets we need
latitude = pn.widgets.FloatInput(name='Latitude', value=5., step=1e-1, 
                                 start=min(df.decimalLatitude), end=max(df.decimalLatitude))
longitude = pn.widgets.FloatInput(name='Longitude', value=5., step=1e-1,
                                  start=min(df.decimalLongitude), end=min(df.decimalLongitude))
species = pn.widgets.MultiSelect(name='Species', options=list(df.species.unique()))

event_date = pn.widgets.DateRangeSlider(
    name='Date Range',
    start=min(pd.to_datetime(df['eventDate'])), end=max(pd.to_datetime(df['eventDate'])),
    value=(dt.datetime(2021, 1, 1), dt.datetime(2021, 12, 1)),
)

country = pn.widgets.Select(name='Country', options=list(df.country.unique()))

button = pn.widgets.Button(name='Update Plots')


#### all our plot functions
### the world map view of occurrence data
def occ_plot(df=df):
    fig = px.scatter_geo(df, lat="decimalLatitude", lon='decimalLongitude', color='species')
    ## making the background transparent below
    fig.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)'
    },
    margin=dict(t=0, b=0, l=0, r=0))
    return fig

## create the world scatter plot
plot_scatter = pn.pane.Plotly(occ_plot())


## function for creating the treemap to show specific point wise values 
def create_cards(df, query=None):
    if query:
        ## filter the dataframe to the selected point
        df = df.query(query)
        df = pd.melt(df, value_vars=['avg_radiance', 'is_invasive', 'avg_deg_urban'])
        fig = px.treemap(df, path=['variable'], values='value')
        fig.data[0].textinfo = 'label+value'

        level = 1 # write the number of the last level you have
        lvl_clr = "#a18dd6"
        font_clr = "black"

        fig.data[0]['marker']['colors'] =[lvl_clr for sector in fig.data[0]['ids'] if len(sector.split("/")) == level]
        fig.data[0]['textfont']['color'] = [font_clr  for sector in fig.data[0]['ids'] if len(sector.split("/")) == level]

        fig.data[0]['textfont']['size'] = 30
        return  fig


###instantiate the cards plot
plot_cards = pn.pane.Plotly(create_cards(df, f'decimalLatitude == -22.258903'), width=400, height=400)


## dependent hidden function to run when a point is clicked in the plot_scatter
@pn.depends(plot_scatter.param.click_data, watch=True)
def _update_after_click_on_1(click_data):
    if click_data !=None:
        lat = click_data['points'][0]['lat']
        lon = click_data['points'][0]['lon']
        plot_cards.object = create_cards(df, f'decimalLatitude == {lat}')


@pn.depends(button.param.clicks, watch=True)
def fetch_data(clicks,start="2022-04-04", end="2022-04-04"):
    ##de = "2022-04-04"\
    #end = event_date.value[0].strftime('%Y-%m-%d')
    cols = ['decimalLatitude','decimalLongitude', 'eventDate', 'species']
    if clicks > 0:
        sql = f"""
        SELECT
            *
        FROM `gbif-challenge.airflow_uploads.gbif_combined`
        WHERE DATE(eventDate) BETWEEN "{start}" AND "{end}"
        """
        bq = client.query(sql).to_dataframe() 
        plot_scatter.object = occ_plot(bq)
        plot_cards.object = create_cards(bq, f"decimalLatitude == {bq.loc[0,'decimalLatitude']}")
        return pn.widgets.DataFrame(bq[cols].head(20))
# def fetch_data(start="2022-04-04", end="2022-04-04"):
#     ##de = "2022-04-04"\
#     #end = event_date.value[0].strftime('%Y-%m-%d')
#     cols = ['decimalLatitude','decimalLongitude', 'eventDate', 'species']

#     sql = f"""
#     SELECT
#         *
#     FROM `gbif-challenge.airflow_uploads.gbif_combined`
#     WHERE DATE(eventDate) BETWEEN "{start}" AND "{end}"
#     """
#     bq = client.query(sql).to_dataframe() 
#     plot_scatter.object = occ_plot(bq)
#     plot_cards.object = create_cards(bq, f"decimalLatitude == {bq.loc[0,'decimalLatitude']}")
#     return pn.widgets.DataFrame(bq[cols].head(20))

        
### placeholder histogram plot of species counts
def species_counts(df=df):
    fig = px.histogram(df, y="species",color_discrete_sequence=['#a18dd6'])
    fig.update_layout({
    'plot_bgcolor': 'rgba(0, 0, 0, 0)',
    'paper_bgcolor': 'rgba(0, 0, 0, 0)',
    })
    return fig
    


# button.on_click(fetch_data)

Watcher(inst=Button(name='Update Plots'), cls=<class 'panel.widgets.button.Button'>, fn=<function fetch_data at 0x7fd11808e940>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [34]:
## The main template to render, sidebar for text

template = pn.template.FastGridTemplate(
    title="GBIF Powered by Covariates",
    sidebar=["We are interested bleh bleh bleh",
             pn.pane.Markdown("## Select Filters"), latitude, longitude,
             '', event_date, '', species, '', country, button],
    accent = '#a18dd6', sidebar_width = 330, background_color = '#f5f5f5',
    neutral_color = '#ffffff',
    corner_radius = 15
)

### specify which portion of the main page grid you want to place a plot in

template.main[:3, :6]=pn.Tabs(('GBIF',pn.Column(plot_scatter)),
                              ('Radiance', pn.pane.HTML(HTML('map1.html'), width=600)), dynamic=True)

template.main[:3, 6:10] = pn.Column(plot_cards)

template.main[3:6, :6]= pn.Column('### Species Counts', species_counts, height=400)
template.main[3:6, 6:10]= pn.Column(fetch_data, height=400)




###color examples
##77cb
#'#faad55'
#'#f0a3bc'

In [ ]:
## tells the terminal command to run the template variable as a dashboard

template.servable();

In [ ]:
####extra code

# center_lat = -3.497341
# center_lon = 38.642394

# zoomlevel=15

# get December image, we're using the "avg_rad" band
# viirs2020_01 = ee.ImageCollection("NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG").filterDate("2022-01-01","2022-01-31").select('avg_rad').median()


# initialize our map
# map1 = geemap.Map()
# map1.addLayer(viirs2020_01, {}, "VIIRS-DNB")
#map1.to_html('map1.html')

# def get_map(lat=20.5936832, long=78.962883, zoom_start=5):
#     mm = fm.Map(location=[lat,long], zoom_start=zoom_start)
#     return mm

# map = get_map()

In [ ]:
# button = pn.widgets.Button(name='Update Plots')
# button

FastGridTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] TemplateActions()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [140537123018640] Markdown(str)
    [140537123628368] Markdown(str)
    [140536836275552] FloatInput(end=60.78981, name='Latitude', start=-30.340799, value=5.0)
    [140536836275216] FloatInput(end=-72.87397, name='Longitude', start=-72.87397, value=5.0)
    [140537436337104] Markdown(str)
    [140537436338592] DateRangeSlider(end=Timestamp('2022-05-23 1..., name='Date Range', start=Timestamp('2021-01-01 0..., value=(datetime.datetime(2021, ..., value_end=datetime.datetime(2021, ..., value_start=datetime.datetime(2021, ...)
    [140536836032496] Markdown(str)
    [140537402464864] MultiSelect(name='Species', options=['Chrysocyon brachyurus', ...])
    [140537122507936] Markdown(str)
    [140537122634000] Select(name='Country', options=['Brazil', 'United K...], value='Brazil')
    [140537647210944] Button(name='Update Plots')
    [140537121027744] Tabs(dynamic=True)
        [0] Column
            [0] Plotly(Figure)
        [1] HTML(HTML, width=600)
    [140537680019168] Column
        [0] Plotly(Figure, height=400, sizing_mode='fixed', width=400)
    [140537121029184] Column(height=400)
        [0] Markdown(str)
        [1] Column
            [0] Column()
            [1] Row
                [0] Plotly(Figure, name='interactive00234')
    [140537121026496] Column(height=400)
        [0] Column
            [0] Column()
            [1] Row
                [0] DataFrame(value=    decimalLatitude  ...)